In [420]:
import numpy as np
import importlib
import Layout
import Greedy

import copy 

#Reset libreria
importlib.reload(px)
importlib.reload(Greedy)
importlib.reload(Layout)

def showtime2(path):
    layout=px.read(path)
    #print ("Estado inicial: ")
    #for s in layout.stacks:
    #    print(s)
    #print()
    seleccionada = SSRC(layout)
 
    while seleccionada != -1:
        if verbose:
            for s in layout.stacks: print(s)
            print("\n")
        
        #Se calcula cuantos contenedores deben retirarse
        columna=copy.deepcopy(layout.stacks[seleccionada])
        nb_unsorted_elements = px.remove_until(columna)

        #Se calculan los ranks
        rank = Greedy.get_ranks(layout.stacks[seleccionada])
        if verbose: print("selected:",layout.stacks[seleccionada])

        #Se retiran los elementos de la columna
        SDpp(layout, seleccionada, rank, nb_unsorted_elements)
        
        if verbose:
            for s in layout.stacks: print(s)
            print("\n")

        #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
        for item in layout.stacks[seleccionada]:
            rank.pop(item, None)

        #print(nb_unsorted_elements)
        SFpp(layout,seleccionada,rank, layout.H-len(layout.stacks[seleccionada]) )#nb_unsorted_elements)

        #Se vuelve a seleccionar una columna
        seleccionada = SSRC(layout)

    #print("SELECCIONADA:", seleccionada)
    #print(len(layout.moves),"\t",layout.unsorted_stacks)
    if verbose: 
        for s in layout.stacks: print(s)
    return len(layout.moves)
    #print()

In [452]:
#selector de columnas a desmantelar
def SSRC(layout):
    best_ev = 0; s_o=-1
    for i in range(len(layout.stacks)):
        if layout.is_sorted_stack(i): continue
        prom = sum(layout.stacks[i]) / len(layout.stacks[i]) 
        ev = 10000 - 100*len(layout.stacks[i]) - prom

        if ev > best_ev:
            best_ev = ev
            s_o = i
    return s_o

In [382]:
def SDpp(layout, s_o, rank, n_elements=100):
    capac = Greedy.capacity(layout,s_o) # espacio libre
    
    ss_o = layout.stacks[s_o]
    
    fixed_elements = np.maximum(len(ss_o)-n_elements,0)
    
    while len(layout.stacks[s_o]) > fixed_elements:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        #print(rank[top])
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            if verbose: 
                for s in layout.stacks: print(s)
                print(s_d,(pos - slack) - rank[top],s_o, rank)
            Greedy.fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)

In [450]:
import sys
def SFpp(layout, s_d, rank, n):
    ub = Layout.gvalue(layout.stacks[s_d])
    cont = 0
    for r in rank:
        while True:
            (s, pos) = Greedy.search_highest(layout, r, ub, s_d)  
            #if verbose:
                #print("r,ub:",r,ub)
                #print(layout.stacks[s][pos])
                
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            ub=layout.stacks[s][pos]-1
            
            c=Greedy.force_move(layout, s, pos, s_d)
            
            
            if c == None:    
                if ub<r: 
                    for stack in layout.stacks:
                        print(stack)
                    print(str(ub+1)+ " del stack " +str(s)+ " no se puede volver a colocar en "+str(s_d)+"!!");
                    sys.exit()
            else: 
                cont+=1
                if cont == n: return
                if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1

In [453]:
import Greedy as gr
import importlib
importlib.reload(Greedy)
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("Instancias") if not isfile(join("Instancias", f))]

verbose = False
for f in onlyfiles:
    if f == 'BF' or f== '3-x' : continue
    #print(f)
    nmoves=0
    for i in range(1,41):
        filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
        nmoves += showtime2(filename)    
    print (f,nmoves/ 40.0);

10-10 165.4
10-6 127.35
3-3 13.375
3-4 13.55
3-5 14.45
3-6 15.475
3-7 17.625
3-8 17.875
4-4 23.9
4-5 26.725
4-6 27.375
4-7 31.85
5-10 60.05
5-4 36.1
5-5 37.55
5-6 42.675
5-7 45.3
5-8 51.6
5-9 55.0
6-10 77.525
6-6 56.85


![image](https://i.imgur.com/XsBW9YR.png)

In [459]:
verbose = True
showtime2("Instancias/3-3/data3-3-9.dat")   

[1, 8, 5]
[6, 4, 9]
[2, 3, 7]


selected: [2, 3, 7]
[1, 8, 5, 7, 3]
[6, 4, 9, 2]
[]


[1, 8]
[6, 4, 9]
[7, 5, 3, 2]


selected: [1, 8]
[]
[6, 4, 9, 1]
[7, 5, 3, 2, 8]


[8, 1]
[6, 4, 9]
[7, 5, 3, 2]


selected: [6, 4, 9]
[8, 1, 4, 6]
[]
[7, 5, 3, 2, 9]


[8, 1]
[9, 6, 4]
[7, 5, 3, 2]


18